# Sample callback

This notebook demonstrates the usage of the callback attribute in `pm.sample`. A callback is a function which gets called for every sample from the trace of a chain. The function is called with the trace and the current draw as arguments and will contain all samples for a single trace.

The sampling process can be interrupted by throwing a `KeyboardInterrupt` from inside the callback.

use-cases for this callback include:

 - Stopping sampling when a number of effective samples is reached
 - Stopping sampling when there are too many divergences
 - Logging metrics to external tools (such as TensorBoard)
 
We'll start with defining a simple model

In [1]:
import numpy as np
import pymc3 as pm

X = np.array([1, 2, 3, 4, 5])
y = X * 2 + np.random.randn(len(X))
with pm.Model() as model:
    intercept = pm.Normal("intercept", 0, 10)
    slope = pm.Normal("slope", 0, 10)

    mean = intercept + slope * X
    error = pm.HalfCauchy("error", 1)
    obs = pm.Normal("obs", mean, error, observed=y)

We can then for example add a callback that stops sampling whenever 100 samples are made, regardless of the number of draws set in the `pm.sample`

In [2]:
def my_callback(trace, draw):
    if len(trace) >= 100:
        raise KeyboardInterrupt()


with model:
    trace = pm.sample(tune=0, draws=500, callback=my_callback, chains=1)

print(len(trace))

<ipython-input-2-e34bf7c63840>:7: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(tune=0, draws=500, callback=my_callback, chains=1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [error, slope, intercept]


Sampling 1 chain for 0 tune and 100 draw iterations (0 + 100 draws total) took 1 seconds.
The chain contains only diverging samples. The model is probably misspecified.
The acceptance probability does not match the target. It is 0.0, but should be close to 0.8. Try to increase the number of tuning steps.
Only one chain was sampled, this makes it impossible to run some convergence checks


100


Something to note though, is that the trace we get passed in the callback only correspond to a single chain. That means that if we want to do calculations over multiple chains at once, we'll need a bit of machinery to make this possible.

In [3]:
def my_callback(trace, draw):
    if len(trace) % 100 == 0:
        print(len(trace))


with model:
    trace = pm.sample(tune=0, draws=500, callback=my_callback, chains=2, cores=2)

<ipython-input-3-1ae38bbf8cec>:7: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(tune=0, draws=500, callback=my_callback, chains=2, cores=2)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [error, slope, intercept]


100
200
300
400
500
100
200
300
400


Sampling 2 chains for 0 tune and 500 draw iterations (0 + 1_000 draws total) took 8 seconds.


500


The estimated number of effective samples is smaller than 200 for some parameters.


We can use the `draw.chain` attribute to figure out which chain the current draw and trace belong to. Combined with some kind of convergence statistic like r_hat we can stop when we have converged, regardless of the amount of specified draws.

In [4]:
import arviz as az


class MyCallback:
    def __init__(self, every=1000, max_rhat=1.05):
        self.every = every
        self.max_rhat = max_rhat
        self.traces = {}

    def __call__(self, trace, draw):
        if draw.tuning:
            return

        self.traces[draw.chain] = trace
        if len(trace) % self.every == 0:
            multitrace = pm.backends.base.MultiTrace(list(self.traces.values()))
            if pm.stats.rhat(multitrace).to_array().max() < self.max_rhat:
                raise KeyboardInterrupt


with model:
    trace = pm.sample(tune=1000, draws=100000, callback=MyCallback(), chains=2, cores=2)

<ipython-input-4-132c122ca424>:22: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(tune=1000, draws=100000, callback=MyCallback(), chains=2, cores=2)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [error, slope, intercept]


arviz - WARNING - Shape validation failed: input_shape: (1, 2000), minimum_shape: (chains=2, draws=4)
arviz - WARNING - Shape validation failed: input_shape: (1, 3000), minimum_shape: (chains=2, draws=4)
/Users/benjamv/opt/miniconda3/envs/pymc3_stable/lib/python3.9/site-packages/arviz/data/io_pymc3.py:265: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data[var_name] = np.array(
/Users/benjamv/opt/miniconda3/envs/pymc3_stable/lib/python3.9/site-packages/arviz/data/io_pymc3.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data[name] = np.array(sel

ValueError: could not broadcast input array from shape (1940,5) into shape (4000,5)

In [5]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Thu Jun 02 2022

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

sys  : 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]
numpy: 1.21.2
pymc3: 3.11.2
arviz: 0.11.2

Watermark: 2.2.0

